## The Annotated Hellogame

This game uses the example of the `hellogame`, which is part of the `clemgame` distribution (but not the benchmark), to explain how to add a game to the benchmark. This game makes use of all abstract classes, which means that it uses default ways of doing the required things as much as possible. This is not strictly necessary, and if you have good reasons, you can do more "manually" (meaning, define code in your game rather than inherit it), but then you have the responsibility to making sure that everything is loaded and logged in the way that the evaluation scripts expect. In any case, it will help getting familiar with the setup to try to understand what's going on here first.

In [3]:
import string
import sys
from typing import Dict, List

In [4]:
sys.path.append('/Users/das/work/local/Gits/2024/clembench-das')

In [36]:
from backends import Model, CustomResponseModel, ModelSpec, load_model_registry, get_model_for
from clemgame.clemgame import GameMaster, GameBenchmark, Player, DialogueGameMaster
from clemgame import get_logger

In [35]:
load_model_registry()

In [6]:
logger = get_logger(__name__)

Let's recall what we did in the previous notebook to define a dialogue game and to simulate players using language models:

- We designed a number of initial prompts that "explain" the game and the role for each given player. In particular, this prompt explained the *move rules* (the format that we expect the player's response to be in), and the *game rules* (what the consequences of a valid move will be, in terms of getting closer to the goal of winning [or not])
- We formalised these rules in python code, so that the `GameMaster` can actually check them, and adjudicate whether the game can continue or not.
- We set up a simple loop that, in turn, prompted each player with the current state of the game, checked the response, updated the state of the game, and did the same for the next player. And so on, until some condition is met -- e.g., success has been reached, or a maximal number of turns has been produced. Or, of course, something has failed in between.

This notebook now takes the step from designing this simple loop to doing so in a way that the game can be fit into our benchmark setup. In this setup, we want to be able to define in advance a number of *game instances* (e.g., in a guessing game, targets to guess), and want a single, standardised entry point for running the game, logging the results, and producing game specific scores per episode, that will ultimately enter into the overall benchmark. For this, the `clemgame` framework sets up *a lot* of scaffolding. Here, we'll try to unpack this a little bit.

Here's the code of the Hellogame:


In [8]:
GAME_NAME = "hellogame"

The main part, if you will, is the game specific `GameMaster`:

In [9]:
class HelloGame(DialogueGameMaster):
    """This class implements a greeting game in which player A
    is greeting another player with a target name.
    """

    def __init__(self, experiment: Dict, player_models: List[Model]):
        super().__init__(GAME_NAME, experiment, player_models)
        self.language: int = experiment["language"]  # fetch experiment parameters here
        self.turns = []
        self.required_words = ["welcome", "hello"]
        self.success = True

    def _on_setup(self, **game_instance):
        self.game_instance = game_instance  # fetch game parameters here

        # Create the players
        self.greeted = Greeted(game_instance["target_name"])
        self.greeter = Greeter(self.player_models[0])

        # Add the players: these will be logged to the records interactions.json
        # Note: During game play the players will be called in the order added here
        self.add_player(self.greeter)
        self.add_player(self.greeted)

        self.required_words.append(self.greeted.name.lower())

    def _on_before_game(self):
        # Do something before the game start e.g. add the initial prompts to the message list for the players
        self.add_user_message(self.greeter, self.game_instance["prompt"])

    def _does_game_proceed(self):
        # Determine if the game should proceed. This is also called once initially.
        if len(self.turns) == 0:
            return True
        return False

    def _validate_player_response(self, player: Player, utterance: str) -> bool:
        # Check responses for specific players
        if player == self.greeter:
            # Check rule: utterance starts with key word
            if not utterance.startswith("GREET:"):
                self.success = False
                return True
            # Check rule: required words are included
            utterance = utterance.lower()
            utterance = utterance.translate(str.maketrans("", "", string.punctuation))
            for required_word in self.required_words:
                if required_word not in utterance:
                    self.success = False
        return True

    def _on_after_turn(self, turn_idx: int):
        self.turns.append(self.success)

    def _after_add_player_response(self, player: Player, utterance: str):
        if player == self.greeter:
            self.add_user_message(self.greeted, utterance)

    def compute_scores(self) -> None:
        score = 0
        if self.success:
            score = 1
        self.log_episode_score('Accuracy', score)

But where's the loop? Turns out, since this loop is very generic, unless we have reasons to define it specifically for this game, we don't have to. It's part of the parent object (`DialogueGameMaster`). Let's have a look at the relevant code of that:

```python
def play(self) -> None:
    self._on_before_game()
    inner_break = False
    while not inner_break and self._does_game_proceed():
        self.log_next_turn()  # not sure if we want to do this always here (or add to _on_before_turn)
        self._on_before_turn(self.current_turn)
        self.logger.info(f"{self.name}: %s turn: %d", self.name, self.current_turn)
        for player in self.__player_sequence():
            if not self._does_game_proceed():
                inner_break = True  # break outer loop without calling _does_game_proceed again
                break  # potentially stop in between player turns
            self.prompt(player)
            while self._should_reprompt(player):
                self._on_before_reprompt(player)
                self.prompt(player, is_reprompt=True)
        self._on_after_turn(self.current_turn)
        self.current_turn += 1
    self._on_after_game()
```

This looks pretty similar to our loop from the previous notebook. What we can see here is that the abstract `GameMaster` method defines various *hooks*, various places in the loop at which we might want something to happen. By default, certain things happen at these positions -- that's what the abstract class defines. If a specific game wants something specific to happen, it needs to overwrite these default methods.

Our code above defines:

- `_on_setup()`:  Stuff to do when first called. Most importantly, this instantiates the objects that are the actual players. It does so based on the list of models that it has been passed; potentially, this means that we have an object at hand here that has a `generate_response()` method (e.g., an LLM).
- `_on_before_game()`: This is being done for every game instance.
- `_does_game_proceed()`: A test function; here, the game should proceed if no turn has been produced yet. In more complex games, this could define a maximal number of turns.
- ` _validate_player_response()`: In this particular implementation, this realises both the move rule (does the reply start with `GREET:`?) and the game rule (does the response contain the target words?).
- `_on_after_turn()`: If we've made it past the first turn, in this particular super simple game, we've already succeeded.
- `_after_add_player_response()`

We'll see in a bit what these do. Let's now first try to get us to a state where we can instantiate this `GameMaster` and can actually play an instance of the game.

First, we need to define what the players do. We have one player that is an actual `Model` (the greeter), and another model which is programmatic (making use of the generic `CustomResponseModel`).

In [10]:
class Greeted(Player):

    def __init__(self, name):
        super().__init__(CustomResponseModel())
        self.name = name

    def _custom_response(self, messages, turn_idx):
        return f"{self.name}: Hi, thanks for having me!"


class Greeter(Player):

    def __init__(self, model: Model):
        super().__init__(model)

    def _custom_response(self, messages, turn_idx):
        raise NotImplementedError("This should not be called, but the remote APIs.")

Next, we also need to specify how the game fits into the overall benchmark. This defines a standard way (a factory) for getting at the required game master, when the overall benchmark is called. (FIXME: We could skip this here and directly initialise `HelloGame()`?)

In [11]:
class HelloGameBenchmark(GameBenchmark):

    def __init__(self):
        super().__init__(GAME_NAME)

    def get_description(self):
        return "Hello game between a greeter and a greeted player"

    def create_game_master(self, experiment: Dict, player_models: List[Model]) -> GameMaster:
        return HelloGame(experiment, player_models)

We can learn a bit about what's happening by looking at the object at various stages of the setup. First, let's look at the object when it's freshly created. (Never mind for now why it needs to know something about "language".)

In [59]:
hgm = hgb.create_game_master({"language": "en"}, [])

In [60]:
hgm.__dict__

{'name': 'hellogame',
 'logger': <Logger __main__ (INFO)>,
 'log_current_turn': -1,
 'interactions': {'players': {}, 'turns': []},
 'requests': [],
 'experiment': {'language': 'en'},
 'player_models': [],
 'players_by_names': OrderedDict(),
 'messages_by_names': {},
 'current_turn': 0,
 'language': 'en',
 'turns': [],
 'required_words': ['welcome', 'hello'],
 'success': True}

Let's give the factory method a more real experiment specification. One thing we notice here is that the initial prompt, which explains the game, is part of the experiment specification, and even of each game instance. This might be surprising at first, because it seems like this could be something that is hard coded into the game (since it goes together with the move and game rules, which are hard coded in the game). However, in the interest of being able to localise this more easily, we've decided to keep this language material outside of the code.

(FIXME: But there are stil english words hard-coded into the gamemaster, like the formatting prefix. Ideally, this should also come from the outside!)

In [61]:
this_experiment = {
      "name": "greet_en",
      "game_instances": [
        {
          "game_id": 0,
          "prompt": "Your task is to greet and happily welcome the other person with the name:\n\nPeter\n\nRules:\n\n1. You must start your message with 'GREET:'\n2. Your message must include 'Hello', 'welcome' and the other person's name\n\nImportant: You only have one try.\n\nLet's start.",
          "target_name": "Peter"
        } ],
      "language": "en"
    }

In [62]:
# hgb.run??
# this is where the sequence of actions leading to play is from...

In [63]:
# [method_name for method_name in dir(hg) if callable(getattr(hg, method_name)) and not method_name.startswith('__')]

In [64]:
THIS_MODEL = 'gpt-4o-mini-2024-07-18'
llm = get_model_for(THIS_MODEL)
llm.set_gen_args(temperature = 0.0, max_tokens= 100) 

hgm = hgb.create_game_master(this_experiment, [llm])

In [65]:
hgm.__dict__

{'name': 'hellogame',
 'logger': <Logger __main__ (INFO)>,
 'log_current_turn': -1,
 'interactions': {'players': {}, 'turns': []},
 'requests': [],
 'experiment': {'name': 'greet_en',
  'game_instances': [{'game_id': 0,
    'prompt': "Your task is to greet and happily welcome the other person with the name:\n\nPeter\n\nRules:\n\n1. You must start your message with 'GREET:'\n2. Your message must include 'Hello', 'welcome' and the other person's name\n\nImportant: You only have one try.\n\nLet's start.",
    'target_name': 'Peter'}],
  'language': 'en'},
 'player_models': [gpt-4o-mini-2024-07-18],
 'players_by_names': OrderedDict(),
 'messages_by_names': {},
 'current_turn': 0,
 'language': 'en',
 'turns': [],
 'required_words': ['welcome', 'hello'],
 'success': True}

This didn't do much, it just filled the experiment specification into an instance of the object.

But the next step does something real. We're calling the `setup()` method (which the game master has inherited from the parent):

In [66]:
hgm.setup(**this_experiment['game_instances'][0])
hgm.__dict__

{'name': 'hellogame',
 'logger': <Logger __main__ (INFO)>,
 'log_current_turn': -1,
 'interactions': {'players': OrderedDict([('GM', 'Game master for hellogame'),
               ('Player 1', 'Greeter, gpt-4o-mini-2024-07-18'),
               ('Player 2', 'Greeted, programmatic')]),
  'turns': []},
 'requests': [],
 'experiment': {'name': 'greet_en',
  'game_instances': [{'game_id': 0,
    'prompt': "Your task is to greet and happily welcome the other person with the name:\n\nPeter\n\nRules:\n\n1. You must start your message with 'GREET:'\n2. Your message must include 'Hello', 'welcome' and the other person's name\n\nImportant: You only have one try.\n\nLet's start.",
    'target_name': 'Peter'}],
  'language': 'en'},
 'player_models': [gpt-4o-mini-2024-07-18],
 'players_by_names': OrderedDict([('Player 1',
               <__main__.Greeter at 0x28ac01c00>),
              ('Player 2', <__main__.Greeted at 0x28b679fc0>)]),
 'messages_by_names': {'Player 1': [], 'Player 2': []},
 'current_

With this, it's time to actually play this game instance. Let's look at what this does to the object.

In [67]:
hgm.play()
hgm.__dict__

{'name': 'hellogame',
 'logger': <Logger __main__ (INFO)>,
 'log_current_turn': 0,
 'interactions': {'players': OrderedDict([('GM', 'Game master for hellogame'),
               ('Player 1', 'Greeter, gpt-4o-mini-2024-07-18'),
               ('Player 2', 'Greeted, programmatic')]),
  'turns': [[{'from': 'GM',
     'to': 'Player 1',
     'timestamp': '2024-08-23T22:45:53.292964',
     'action': {'type': 'send message',
      'content': "Your task is to greet and happily welcome the other person with the name:\n\nPeter\n\nRules:\n\n1. You must start your message with 'GREET:'\n2. Your message must include 'Hello', 'welcome' and the other person's name\n\nImportant: You only have one try.\n\nLet's start."}},
    {'from': 'Player 1',
     'to': 'GM',
     'timestamp': '2024-08-23T22:45:54.074230',
     'action': {'type': 'get message',
      'content': "GREET: Hello Peter! Welcome! I'm so glad to have you here!"}},
    {'from': 'GM',
     'to': 'Player 2',
     'timestamp': '2024-08-23T22:4

Quite a lot! (And note that we wrote very little code to make all of this happen, thanks to the code that was inherited (and that is the same for all, or at least many, games).)

Let's unpack this a bit:

Who has played?

In [71]:
hgm.interactions['players']

OrderedDict([('GM', 'Game master for hellogame'),
             ('Player 1', 'Greeter, gpt-4o-mini-2024-07-18'),
             ('Player 2', 'Greeted, programmatic')])

We note that the game master has a real presence in the game. This is even more clear when we look at the turns:

In [72]:
hgm.interactions['turns']

[[{'from': 'GM',
   'to': 'Player 1',
   'timestamp': '2024-08-23T22:45:53.292964',
   'action': {'type': 'send message',
    'content': "Your task is to greet and happily welcome the other person with the name:\n\nPeter\n\nRules:\n\n1. You must start your message with 'GREET:'\n2. Your message must include 'Hello', 'welcome' and the other person's name\n\nImportant: You only have one try.\n\nLet's start."}},
  {'from': 'Player 1',
   'to': 'GM',
   'timestamp': '2024-08-23T22:45:54.074230',
   'action': {'type': 'get message',
    'content': "GREET: Hello Peter! Welcome! I'm so glad to have you here!"}},
  {'from': 'GM',
   'to': 'Player 2',
   'timestamp': '2024-08-23T22:45:54.074723',
   'action': {'type': 'send message',
    'content': "GREET: Hello Peter! Welcome! I'm so glad to have you here!"}},
  {'from': 'Player 2',
   'to': 'GM',
   'timestamp': '2024-08-23T22:45:54.074809',
   'action': {'type': 'get message',
    'content': 'Peter: Hi, thanks for having me!'}}]]

It is the game master who initiiates the game by prompting player 1, who receives the response from that player, and who then (because the message passed the check), prompts player 2 to respond, and takes their response. Since this is a single-turn game, the game ends here. And because the required words were spoken, the success is:

In [73]:
hgm.success

True

For now, this shall suffice. What this hopefully has made clear is that the basic structure (prompt that verbalises rules, python code that checks rules) is there, if hidden a little bit behind many abstractions. But a useful next step from here could be to take this as a template to implement a game that does not stray far from this pattern, investigating in the process more what the instance generator does (for this, see the notebook `how_to_add_games_example.ipynb` (FIXME: which, however, is a little bit older and does not fully follow this pattern; e.g., it inherits directly from `GameMaster`, instead of from `DialogGameMaster`.)